In [ ]:
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyfits as fits
from scipy.stats import norm

In [ ]:
import matplotlib.animation as animation
%pylab inline

In [ ]:
docompute = True

In [ ]:
root_path = '/home/apps/astro/data/muse/'

In [ ]:
#filename = "muse_scipost_N_ASASSN14lp_1_new_DATACUBE_FINAL_IFU_004_zap_all.fits"
filename = "muse_scipost_N_SN2009bb_DATACUBE_FINAL_IFU_001_zap.fits"
#filename = "muse_scipost_N_SN2004ef_DATACUBE_FINAL_IFU_002_skysub.fits"
data = fits.open(root_path+filename)
_image = data[1].data
header = data[1].header
del data
#image = image[:, 63:263, 100:300]
image = _image[:, 100:300, 20:220]
(nl, nx, ny) = np.shape(image)
image2D = np.sum(image, axis = 0)
image2D[np.invert(np.isfinite(image2D))] = 0
image2D = np.arcsinh(image2D)
fig, ax = plt.subplots(figsize = (10, 10))
l1, l2 = (np.percentile(image2D.flatten(), 1), np.percentile(image2D.flatten(), 99))
print(l1, l2)
color = ax.imshow(image2D, clim = (10, 14));
fig.colorbar(color)


In [ ]:
# spectral wavelengths
CRVAL3 = float(header["CRVAL3"])
CRPIX3 = float(header["CRPIX3"])
CUNIT3 = header["CUNIT3"]
CD31 = float(header["CD3_1"])
CD32 = float(header["CD3_2"])
CD33 = float(header["CD3_3"])
NAXIS3 = int(header["NAXIS3"])
lambdas = np.array(CRVAL3 + CD33 * (np.arange(NAXIS3) - CRPIX3))

**Asumimos que $H\alpha$ es la más intensa.**

In [ ]:
H_alpha = 6562.8

In [ ]:
fig, ax = plt.subplots(figsize = (20, 5))
spectra = _image[:, 120, 50]
ax.plot(lambdas, spectra)
ax.set_xlim(min(lambdas), max(lambdas))
plt.ylabel('intensity')
plt.xlabel('wave-lenghts')
plt.plot([H_alpha,H_alpha],[spectra.min(),spectra.max()], c='red')

In [ ]:
_max = lambdas[spectra.argmax()]
_max

In [ ]:
z_1 = _max/H_alpha

In [ ]:
lambdas_shiftted = lambdas/z_1

In [ ]:
fig, ax = plt.subplots(figsize = (20, 5))
spectra = image[:, 120, 50]
ax.plot(lambdas_shiftted, spectra)
ax.set_xlim(min(lambdas), max(lambdas))
plt.ylabel('intensity')
plt.xlabel('wave-lenghts')
plt.plot([H_alpha,H_alpha],[spectra.min(),spectra.max()], c='red')

### Shift of the full data set.

In [ ]:
all_maximums = image.argmax(0)
all_maximums.shape

In [ ]:
all_lambdas = lambdas[all_maximums]
print(all_lambdas.shape)

all_z_1 = all_lambdas/H_alpha
print(all_z_1.shape)

In [ ]:
multiply_lambdas = np.repeat(np.matrix(lambdas), all_z_1.shape[0], axis=0)  #(3681,) to (3681,321,320)

In [ ]:
multiply_lambdas.T.shape

In [ ]:
transformed = np.array([multiply_lambdas for i in range(all_z_1.shape[1])]).T

In [ ]:
transformed.shape

In [ ]:
all_lambdas_shiftted = transformed/all_z_1
all_lambdas_shiftted.shape

In [ ]:
all_lambdas_shiftted.shape

In [ ]:
_minimum = int(all_lambdas.min())
_maximum = int(all_lambdas.max())

_minimum, _maximum

In [ ]:
int_all_lambdas_shiftted = all_lambdas_shiftted.astype(int)

In [ ]:
int_all_lambdas_shiftted.shape

In [ ]:
new_image = np.ones((_maximum-_minimum, image.shape[1], image.shape[2]))

In [ ]:
new_image = new_image * np.nanmean(image, axis=0)

In [ ]:
new_image.shape

In [ ]:
_i = 0
_j = 0
_k = 0

for i in range(int_all_lambdas_shiftted.shape[0]):
    
    _j = 0
    for j in range(int_all_lambdas_shiftted.shape[1]):  
        
        _k = 0
        for k in range(int_all_lambdas_shiftted.shape[2]):
            new_image[i,j,k] = image[_i,_j,_k]
            _k += 1
        _j += 1
    
    i += 1

In [ ]:
new_image.shape